### Topocentric Conversion

In [25]:
## Status: success
## Conversion of geocentric coordinates to topocentric coordinates
## Note: PROJ v8.0 or above only.
## Option `-d 6` declares 6 places decimal number for output

! echo -1130481.8931 6093598.8489 1501830.3669 | \
    /mybin/proj/bin/cct -d 6 +proj=topocentric +ellps=WGS84 +X_0=-1129466.1104 +Y_0=6094053.7967 +Z_0=1500755.5005

  1081.680976    1106.388721     -0.188266       -0.0000


In [27]:
## Inversion of the above
## Note: PROJ v8.0 or above only.

! echo 1081.680976    1106.388721     -0.188266       -0.0000 | \
    /mybin/proj/bin/cct -d 5 +proj=topocentric +ellps=WGS84 +X_0=-1129466.1104 +Y_0=6094053.7967 +Z_0=1500755.5005 +inv

-1130481.89310  6093598.84890  1501830.36690        0.9966


In [28]:
## From long-lat-h-t to ECEF-XYZ

! echo 100.51 13.71 0 2020.0 | cct -d 9 +proj=cart +ellps=WGS84

-1130481.893066850  6093598.848949597  1501830.366850326     2020.0000


In [22]:
## From ECEF-XYZ to ENU (+lat_0=13.71 +lon_0=100.51 +h_0=0)
## Note: PROJ v8.0 or above only.
## Check the validity of the previous result
## Status: OK.

! echo -1130481.893066850 6093598.848949597 1501830.366850326 2020.0000 | \
    /mybin/proj/bin/cct -d 9 +proj=topocentric +ellps=WGS84 +lat_0=13.71 +lon_0=100.51 +h_0=0

## Expect: 0 0 0 0
## At center of ENU, all are zero.

 -0.000000002   -0.000000000  -0.000000000       -0.0000


### Two Step Conversion:- Long-lat-h to ENU

Conversion steps:
- +proj=cart +ellps=WGS84
- +proj=topocentric

#### Geodetic long-lat-h to ENU Conversion

In [46]:
## This converts long-lat-h-t
## to ENU (+lat_0=13.71 +lon_0=100.51 +h_0=0)
## Requires: PROJ v8.0
## Status: OK

! echo 100.51  13.71  0  1998.0  "[Point attribute]" | \
/mybin/proj/bin/cct +proj=pipeline +ellps=WGS84 \
+step +proj=cart \
+step +proj=topocentric +ellps=WGS84 +lat_0=13.71 +lon_0=100.51 +h_0=0

## Expect: 0 0 0 inf [Point attribute]
## At the origin of ENU, coord are zero.

      -0.0000         0.0000       -0.0000           inf [Point attribute]


#### ENU to Geodetic long-lat-h Conversion

How to create the PROJ string?
- based off the direct PROJ string
- reverse the sequence of the steps
- add "+inv" to each "+step"

In [45]:
## Inversion of the previous code cell
## Steps are reversed, and `+inv` is add in each step

! echo 0  0  0 | \
/mybin/proj/bin/cct +proj=pipeline +ellps=WGS84 \
+step +proj=topocentric +ellps=WGS84 +lat_0=13.71 +lon_0=100.51 +h_0=0 +inv \
+step +proj=cart +inv

## Expect: 100.51 13.71 0
## Status: almost perfect, (time component ?)

100.5100000000   13.7100000000        0.0000        0.9966


---

In [50]:
## At  lon_0 + 0.01,  lat_0 + 0.01
## E and N will be around 110 meters

! echo 100.511  13.711  0 | \
/mybin/proj/bin/cct -d 8 +proj=pipeline +ellps=WGS84 \
+step +proj=cart \
+step +proj=topocentric +ellps=WGS84 +lat_0=13.71 +lon_0=100.51 +h_0=0

 108.16763222   110.63690404   -0.00188253           inf


In [51]:
## Inversion check of the previous code cell
## Status: OK

! echo 108.16763222   110.63690404   -0.00188253 | \
/mybin/proj/bin/cct +proj=pipeline +ellps=WGS84 \
+step +proj=topocentric +ellps=WGS84 +lat_0=13.71 +lon_0=100.51 +h_0=0 +inv \
+step +proj=cart +inv

## Expect: 100.511  13.711  0
## Forget the 4th component!

100.5110000000   13.7110000000        0.0000        0.9966


---

In [31]:
! cct -h

--------------------------------------------------------------------------------
Usage: cct [-options]... [+operator_specs]... infile...
--------------------------------------------------------------------------------
Options:
--------------------------------------------------------------------------------
    -c x,y,z,t        Specify input columns for (up to) 4 input parameters.
                      Defaults to 1,2,3,4
    -d n              Specify number of decimals in output.
    -I                Do the inverse transformation
    -o /path/to/file  Specify output file name
    -t value          Provide a fixed t value for all input data (e.g. -t 0)
    -z value          Provide a fixed z value for all input data (e.g. -z 0)
    -s n              Skip n first lines of a infile
    -v                Verbose: Provide non-essential informational output.
                      Repeat -v for more verbosity (e.g. -vv)
-----------------------------------------------------------------------